In [ ]:
import pandas as pd
import utils_comp as utils
import utils_max_flow
import utils_centrality
import pickle

In [ ]:
# Dataset; 'iggin' or 'iggielgn'
dataset = 'iggielgn'

# S-t-flow; 's-t-degree' or 's-t-type'
s_t_flow = 'hybrid'

### Load data from 'N-k_CENTRALITY.ipynb' and 'N-k_MAX_FLOW.ipynb'

In [ ]:
centrality_random_node_removal_df = pd.read_pickle('results/centrality/iggielgn/random_node_removal.pkl')
centrality_greedy_node_removal_df = pd.read_pickle('results/centrality/iggielgn/greedy_node_removal.pkl')

max_flow_random_node_removal_df = pd.read_pickle('results/max_flow/iggielgn/hybrid/random_node_removal_df.pkl')
max_flow_greedy_node_removal_df = pd.read_pickle('results/max_flow/iggielgn/hybrid/wfcr_node_removal_df.pkl')

centrality_random_edge_removal_df = pd.read_pickle('results/centrality/iggielgn/random_edge_removal.pkl')
centrality_greedy_edge_removal_df = pd.read_pickle('results/centrality/iggielgn/greedy_edge_removal.pkl')

max_flow_random_edge_removal_df = pd.read_pickle('results/max_flow/iggielgn/hybrid/random_edge_removal_df.pkl')
max_flow_greedy_edge_removal_df = pd.read_pickle('results/max_flow/iggielgn/hybrid/wfcr_edge_removal_df.pkl')

In [ ]:
with open('graph_objects/G_simple_directed_iggielgn.pickle', 'rb') as f:
    G_simple_directed = pickle.load(f)
    G_simple_directed.name = 'G_simple_directed' 

# Direct comparison of index 

### Node

In [ ]:
utils.plot_direct_comparison(centrality_greedy_node_removal_df, centrality_random_node_removal_df, max_flow_greedy_node_removal_df, max_flow_random_node_removal_df)

In [ ]:
utils.common_entities(centrality_greedy_node_removal_df, max_flow_greedy_node_removal_df)

### Edge

In [ ]:
utils.plot_direct_comparison(centrality_greedy_edge_removal_df, centrality_random_edge_removal_df, max_flow_greedy_edge_removal_df, max_flow_random_edge_removal_df)

In [ ]:
# Ignore SettingWithCopyWarning
pd.options.mode.chained_assignment = None

corrected_centrality_greedy_edge_removal_df = centrality_greedy_edge_removal_df.iloc[1:]
corrected_centrality_greedy_edge_removal_df['removed_entity'] = corrected_centrality_greedy_edge_removal_df['removed_entity'].apply(lambda edge: tuple(edge))
for i in range(len(corrected_centrality_greedy_edge_removal_df)):
    if corrected_centrality_greedy_edge_removal_df['removed_entity'].iloc[i] not in G_simple_directed.edges():
        corrected_centrality_greedy_edge_removal_df['removed_entity'].iloc[i] = (corrected_centrality_greedy_edge_removal_df['removed_entity'].iloc[i][1], corrected_centrality_greedy_edge_removal_df['removed_entity'].iloc[i][0])

corrected_max_flow_greedy_edge_removal_df = max_flow_greedy_edge_removal_df.iloc[1:]
corrected_max_flow_greedy_edge_removal_df['removed_entity'] = corrected_max_flow_greedy_edge_removal_df['removed_entity'].apply(lambda edge: tuple(edge))
for i in range(len(corrected_max_flow_greedy_edge_removal_df)):
    if corrected_max_flow_greedy_edge_removal_df['removed_entity'].iloc[i] not in G_simple_directed.edges():
        corrected_max_flow_greedy_edge_removal_df['removed_entity'].iloc[i] = (corrected_max_flow_greedy_edge_removal_df['removed_entity'].iloc[i][1], corrected_max_flow_greedy_edge_removal_df['removed_entity'].iloc[i][0])
        

In [ ]:
utils.common_entities(corrected_centrality_greedy_edge_removal_df, corrected_max_flow_greedy_edge_removal_df)

# Transformation between 'domains'

In [ ]:
centrality_greedily_removed_nodes_lst = centrality_greedy_node_removal_df['removed_entity'].dropna().tolist()
max_flow_greedily_removed_nodes_lst = max_flow_greedy_node_removal_df['removed_entity'].dropna().tolist()

centrality_greedily_removed_edges_lst = centrality_greedy_edge_removal_df['removed_entity'].dropna().tolist()
max_flow_greedily_removed_edges_lst = max_flow_greedy_edge_removal_df['removed_entity'].dropna().tolist()

In [ ]:
# Correct directionality of edges
centrality_greedily_removed_edges_lst = utils.correct_edges([tuple(e) for e in centrality_greedily_removed_edges_lst])
max_flow_greedily_removed_edges_lst = utils.correct_edges([tuple(e) for e in max_flow_greedily_removed_edges_lst])

In [ ]:
# Limit to 100 removed entities
centrality_greedy_node_removal_df = centrality_greedy_node_removal_df.iloc[:101]
centrality_greedy_edge_removal_df = centrality_greedy_edge_removal_df.iloc[:101]

### Max flow based N-k analysis employing greedy entity removal order resulting from centrality analysis

In [ ]:
centrality_greedily_removed_nodes_max_flow_df = utils_max_flow.flow_capacity_robustness(G_simple_directed, heuristic='greedy_centrality', remove='node', greedy_centrality_lst=centrality_greedily_removed_nodes_lst)
centrality_greedily_removed_nodes_max_flow_df.to_pickle('results/comparison/centrality_greedily_removed_nodes_max_flow_df.pkl')
centrality_greedily_removed_nodes_max_flow_df = pd.read_pickle('results/comparison/centrality_greedily_removed_nodes_max_flow_df.pkl')

In [ ]:
centrality_greedily_removed_edges_max_flow_df = utils_max_flow.flow_capacity_robustness(G_simple_directed, heuristic='greedy_centrality', remove='edge', greedy_centrality_lst=centrality_greedily_removed_edges_lst)
centrality_greedily_removed_edges_max_flow_df.to_pickle('results/comparison/centrality_greedily_removed_edges_max_flow_df.pkl')
centrality_greedily_removed_edges_max_flow_df = pd.read_pickle('results/comparison/centrality_greedily_removed_edges_max_flow_df.pkl')

In [ ]:
""" fig = utils.plot_transform_analysis([centrality_greedily_removed_nodes_max_flow_df, centrality_greedily_removed_edges_max_flow_df], [max_flow_greedy_node_removal_df, max_flow_greedy_edge_removal_df], 'capacity_robustness_max_flow')
fig.savefig('saved_plots/'+dataset+'/'+s_t_flow+'/comparison/max_flow_employing_centrality_removal_order.png') """

In [ ]:
""" utils.common_entities(centrality_greedily_removed_nodes_max_flow_df, max_flow_greedy_node_removal_df) """

In [ ]:
""" utils.common_entities(centrality_greedily_removed_edges_max_flow_df, max_flow_greedy_edge_removal_df) """

### Centrality based N-k analysis employing greedy entity removal order resulting from max flow analysis

In [ ]:
max_flow_greedily_removed_nodes_centrality_df, _ = utils_centrality.n_minus_k(G_simple_directed, heuristic='max_flow', remove='node', greedy_max_flow_lst=max_flow_greedily_removed_nodes_lst)
max_flow_greedily_removed_nodes_centrality_df.to_pickle('results/comparison/max_flow_greedily_removed_nodes_centrality_df.pkl')
max_flow_greedily_removed_nodes_centrality_df = pd.read_pickle('results/comparison/max_flow_greedily_removed_nodes_centrality_df.pkl')

In [ ]:
max_flow_greedily_removed_edges_centrality_df, _ = utils_centrality.n_minus_k(G_simple_directed, heuristic='max_flow', remove='edge', greedy_max_flow_lst=max_flow_greedily_removed_edges_lst)
max_flow_greedily_removed_edges_centrality_df.to_pickle('results/comparison/max_flow_greedily_removed_edges_centrality_df.pkl')
max_flow_greedily_removed_edges_centrality_df = pd.read_pickle('results/comparison/max_flow_greedily_removed_edges_centrality_df.pkl')

In [ ]:
""" fig = utils.plot_transform_analysis([max_flow_greedily_removed_nodes_centrality_df, max_flow_greedily_removed_edges_centrality_df], [centrality_greedy_node_removal_df, centrality_greedy_edge_removal_df], 'composite')
fig.savefig('saved_plots/'+dataset+'/'+s_t_flow+'/comparison/centrality_employing_max_flow_removal_order.png') """

In [ ]:
""" utils.common_entities(max_flow_greedily_removed_nodes_centrality_df, centrality_greedy_node_removal_df) """

In [ ]:
""" utils.common_entities(max_flow_greedily_removed_edges_centrality_df, centrality_greedy_edge_removal_df) """